# Loading datasets from database (only user and post_text)

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

DB_URL = os.getenv("DB_URL")

if not DB_URL:
    raise ValueError("Database URL not found. Make sure you have a .env file or set the environment variable.")

user_data = pd.read_sql(
    """SELECT * FROM public.user_data""",
    con=DB_URL
)

user_data

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [4]:
post_text_df = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    con=DB_URL
)

post_text_df

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


# Changing post_text - creating features

In [7]:
post_text_df['post_length'] = post_text_df['text'].apply(len)

In [9]:
post_text_df['num_hashtags'] = post_text_df['text'].apply(lambda x: x.count('#'))

In [11]:
post_text_df['num_mentions'] = post_text_df['text'].apply(lambda x: x.count('@'))

In [13]:
post_text_df['has_hashtag'] = post_text_df['text'].apply(lambda x: '#' in x)

In [15]:
post_text_df['has_url'] = post_text_df['text'].apply(lambda x: 'http' in x)

In [17]:
post_text_df

,post_id,text,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url
0,1,UK economy facing major risks\n\nThe UK manufa...,business,1967,0,0,False,False
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,2701,0,0,False,False
2,3,Asian quake hits European shares\n\nShares in ...,business,3408,0,0,False,False
3,4,India power shares jump on debut\n\nShares in ...,business,1026,0,0,False,False
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,889,0,0,False,False
...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,803,0,0,False,False
7019,7316,I give this movie 2 stars purely because of it...,movie,800,0,0,False,False
7020,7317,I cant believe this film was allowed to be mad...,movie,636,0,0,False,False
7021,7318,The version I saw of this film was the Blockbu...,movie,728,0,0,False,False


In [19]:
import numpy as np

post_text_df.loc[:,post_text_df.dtypes==np.object_].columns

Index(['text', 'topic'], dtype='object')

In [21]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(DB_URL)


post_text_df.to_sql('daria_luzina_features_post_2', con=engine, index=False) # записываем таблицу

df = pd.read_sql('SELECT * FROM daria_luzina_features_post_2', con=engine)

In [23]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(DB_URL)

df = pd.read_sql('SELECT * FROM daria_luzina_features_post_2', con=engine)

In [25]:
df

,post_id,text,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url
0,1,UK economy facing major risks\n\nThe UK manufa...,business,1967,0,0,False,False
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,2701,0,0,False,False
2,3,Asian quake hits European shares\n\nShares in ...,business,3408,0,0,False,False
3,4,India power shares jump on debut\n\nShares in ...,business,1026,0,0,False,False
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,889,0,0,False,False
...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,803,0,0,False,False
7019,7316,I give this movie 2 stars purely because of it...,movie,800,0,0,False,False
7020,7317,I cant believe this film was allowed to be mad...,movie,636,0,0,False,False
7021,7318,The version I saw of this film was the Blockbu...,movie,728,0,0,False,False


In [27]:
import pandas as pd
from sqlalchemy import create_engine


def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(DB_URL)
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [29]:
def load_features() -> pd.DataFrame:
    query = "SELECT * FROM public.user_data"
    return batch_load_sql(query)


def load_features() -> pd.DataFrame:
    df_users = batch_load_sql("SELECT * from public.user_data")
    df_posts = batch_load_sql("SELECT * from daria_luzina_features_post_2")
    return df_users, df_posts

df_users, df_posts = load_features()

In [30]:
df_users

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [33]:
df_posts

,post_id,text,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url
0,1,UK economy facing major risks\n\nThe UK manufa...,business,1967,0,0,False,False
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,2701,0,0,False,False
2,3,Asian quake hits European shares\n\nShares in ...,business,3408,0,0,False,False
3,4,India power shares jump on debut\n\nShares in ...,business,1026,0,0,False,False
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,889,0,0,False,False
...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,803,0,0,False,False
7019,7316,I give this movie 2 stars purely because of it...,movie,800,0,0,False,False
7020,7317,I cant believe this film was allowed to be mad...,movie,636,0,0,False,False
7021,7318,The version I saw of this film was the Blockbu...,movie,728,0,0,False,False


In [35]:
dataset = df_users[df_users['user_id']==200]

In [37]:
dataset = dataset.merge(df_posts.drop(columns='text'), how='cross')

In [39]:
dataset

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url
0,200,1,34,Russia,Degtyarsk,3,Android,ads,1,business,1967,0,0,False,False
1,200,1,34,Russia,Degtyarsk,3,Android,ads,2,business,2701,0,0,False,False
2,200,1,34,Russia,Degtyarsk,3,Android,ads,3,business,3408,0,0,False,False
3,200,1,34,Russia,Degtyarsk,3,Android,ads,4,business,1026,0,0,False,False
4,200,1,34,Russia,Degtyarsk,3,Android,ads,5,business,889,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,200,1,34,Russia,Degtyarsk,3,Android,ads,7315,movie,803,0,0,False,False
7019,200,1,34,Russia,Degtyarsk,3,Android,ads,7316,movie,800,0,0,False,False
7020,200,1,34,Russia,Degtyarsk,3,Android,ads,7317,movie,636,0,0,False,False
7021,200,1,34,Russia,Degtyarsk,3,Android,ads,7318,movie,728,0,0,False,False


In [41]:
dataset = dataset.drop(columns = 'user_id')

In [43]:
dataset = dataset.set_index('post_id')

In [45]:
dataset

,gender,age,country,city,exp_group,os,source,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url
post_id,,,,,,,,,,,,,
1,1,34,Russia,Degtyarsk,3,Android,ads,business,1967,0,0,False,False
2,1,34,Russia,Degtyarsk,3,Android,ads,business,2701,0,0,False,False
3,1,34,Russia,Degtyarsk,3,Android,ads,business,3408,0,0,False,False
4,1,34,Russia,Degtyarsk,3,Android,ads,business,1026,0,0,False,False
5,1,34,Russia,Degtyarsk,3,Android,ads,business,889,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,1,34,Russia,Degtyarsk,3,Android,ads,movie,803,0,0,False,False
7316,1,34,Russia,Degtyarsk,3,Android,ads,movie,800,0,0,False,False
7317,1,34,Russia,Degtyarsk,3,Android,ads,movie,636,0,0,False,False


In [49]:
import os
from catboost import CatBoostClassifier

def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH


def load_models():
    model_path = get_model_path("/Users/daraluzina/ML/HW_22/Ver_1.0/catboost_model_3.cbm")  
    
    # Load CatBoost model
    model = CatBoostClassifier()
    model.load_model(model_path)
    
    return model

In [51]:
model_test = load_models()

In [53]:
model_test.predict_proba(dataset)[:,1]

array([0.05610598, 0.06030063, 0.05503424, ..., 0.08246341, 0.08307035,
       0.08307035])

In [55]:
dataset['predict_proba'] = model_test.predict_proba(dataset)[:,1]

In [57]:
dataset = dataset.sort_values(by='predict_proba', ascending=False).head(5)

In [59]:
dataset

,gender,age,country,city,exp_group,os,source,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url,predict_proba
post_id,,,,,,,,,,,,,,
4872,1,34,Russia,Degtyarsk,3,Android,ads,movie,2519,0,1,False,False,0.100915
3072,1,34,Russia,Degtyarsk,3,Android,ads,covid,103,6,0,True,True,0.100572
5167,1,34,Russia,Degtyarsk,3,Android,ads,movie,2696,0,1,False,False,0.098239
6487,1,34,Russia,Degtyarsk,3,Android,ads,movie,2034,0,1,False,False,0.095999
5916,1,34,Russia,Degtyarsk,3,Android,ads,movie,877,3,0,True,False,0.095439


In [61]:
dataset = dataset.merge(df_posts, left_index=True, right_on = 'post_id').reset_index()

In [63]:
dataset

,index,gender,age,country,city,exp_group,os,source,topic_x,post_length_x,...,has_url_x,predict_proba,post_id,text,topic_y,post_length_y,num_hashtags_y,num_mentions_y,has_hashtag_y,has_url_y
0,4700,1,34,Russia,Degtyarsk,3,Android,ads,movie,2519,...,False,0.100915,4872,Perhaps Im being too generous when I give this...,movie,2519,0,1,False,False
1,3176,1,34,Russia,Degtyarsk,3,Android,ads,covid,103,...,True,0.100572,3072,Manila returns to lockdown \n\n#lka #world #Ph...,covid,103,6,0,True,True
2,4988,1,34,Russia,Degtyarsk,3,Android,ads,movie,2696,...,False,0.098239,5167,"Contrary to the comment posted directly below,...",movie,2696,0,1,False,False
3,6240,1,34,Russia,Degtyarsk,3,Android,ads,movie,2034,...,False,0.095999,6487,"This is the page for House of Exorcism, but mo...",movie,2034,0,1,False,False
4,5694,1,34,Russia,Degtyarsk,3,Android,ads,movie,877,...,False,0.095439,5916,Sarah Plain and Talls Winters end was the best...,movie,877,3,0,True,False


In [65]:
dataset = dataset[['post_id','text','topic_x']]


In [67]:
dataset

,post_id,text,topic_x
0,4872,Perhaps Im being too generous when I give this...,movie
1,3072,Manila returns to lockdown \n\n#lka #world #Ph...,covid
2,5167,"Contrary to the comment posted directly below,...",movie
3,6487,"This is the page for House of Exorcism, but mo...",movie
4,5916,Sarah Plain and Talls Winters end was the best...,movie


In [69]:
list_of_dicts = dataset.to_dict('records')
list_of_dicts

[{'post_id': 4872,
  'text': 'Perhaps Im being too generous when I give this film two and a half stars out of five, but there was an occasional moment. However, as An American Werewolf movie this one is a missed chance! There are no real plot connections to the superior original to speak of, but the story is similar in some ways to London.*Possible Spoiler Warning* American kids go to foreign country, one falls in love with a beautiful girl. Another one of the kids gets slaughtered by a werewolf in the same night that one gets bitten, and despite his undead friends warnings, by the light of the full moon he sprouts fur, fangs, and claws!But there are some differences in the story, for one; the girl is one of the werewolves. Second; there are three American Kids. And third; theres some weird-@$$ werewolf cult intent on taking over the world! As crazy as it sounds, that last one, WASNT a joke! *Spoiler Ends*The films suffers from many things, first the weak acting drags it down immensely